#### Notebook for running Chain-of-Thought with supporting context experiments 

In [1]:
import sys, os
sys.path.append('..')
root = '../root/'

In [2]:
import joblib
import numpy as np
from agents import CoTAgent, ReflexionStrategy
from util import summarize_trial, log_trial, save_agents

#### Load the HotPotQA Sample

In [3]:
hotpot = joblib.load('../data/hotpot-qa-distractor-sample.joblib').reset_index(drop = True)

hotpot['supporting_paragraphs'] = None
for ind, row in hotpot.iterrows():
    supporting_articles = row['supporting_facts']['title']
    articles = row['context']['title']
    sentences = row['context']['sentences'] 
    supporting_paragraphs = []
    for article in supporting_articles:
        supporting_paragraph = ''.join(sentences[np.where(articles == article)][0])
        supporting_paragraphs.append(supporting_paragraph)
    supporting_paragraphs = '\n\n'.join(supporting_paragraphs)
    hotpot.at[ind, 'supporting_paragraphs'] = supporting_paragraphs

#### Define the Reflexion Strategy

In [4]:
print(ReflexionStrategy.__doc__)


    NONE: No reflection
    LAST_ATTEMPT: Use last reasoning trace in context 
    REFLEXION: Apply reflexion to the next reasoning trace 
    LAST_ATTEMPT_AND_REFLEXION: Use last reasoning trace in context and apply reflexion to the next reasoning trace 
    


In [5]:
strategy: ReflexionStrategy = ReflexionStrategy.REFLEXION

#### Initialize a CoTAgent for each question

In [6]:
# Make a local transformers-based LLM (loads model when first used)
from hf_transformers_llm import HFTransformersLLM
# set HF_LOCAL_MODEL to the model you want, or put the model id here:
local_model = "google/flan-t5-small"
llm = HFTransformersLLM(model_id=local_model, device=None, load_in_4bit=False)

In [7]:
from prompts import cot_agent_prompt, cot_reflect_agent_prompt, cot_reflect_prompt
from fewshots import COT, COT_REFLECT
agents = [CoTAgent(
              row['question'],
              row['supporting_paragraphs'],
              row['answer'],
              agent_prompt=cot_agent_prompt if strategy == ReflexionStrategy.NONE else cot_reflect_agent_prompt,
              cot_examples=COT,
              reflect_prompt=cot_reflect_prompt,
              reflect_examples=COT_REFLECT,
              self_reflect_llm=llm, #add customized llm here
              action_llm=llm, #add customized llm here
          ) for _, row in hotpot.iterrows()]

#### Run `n` trials

In [8]:
n = 5
trial = 0
log = ''

In [9]:
for i in range(n):
    for agent in [a for a in agents if not a.is_correct()]:
        agent.run(reflexion_strategy = strategy)
        print(f'Answer: {agent.key}')
    trial += 1
    log += log_trial(agents, trial)
    correct, incorrect = summarize_trial(agents)
    print(f'Finished Trial {trial}, Correct: {len(correct)}, Incorrect: {len(incorrect)}')

/opt/miniconda3/envs/reflexion/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/miniconda3/envs/reflexion/lib/python3.11/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


HFTransformersLLM: device=cpu, preferred_dtype=torch.float32, load_in_4bit=False


/opt/miniconda3/envs/reflexion/lib/python3.11/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


HFTransformersLLM: loaded model google/flan-t5-small with config.model_type=t5
Thought: Viva channels exist in some European countries; the first spin-offs were launched in Poland and Switzerland in 2000
Action: Viva channels exist in some European countries; the first spin-offs were launched in Poland and Switzerland in 2000
Invalid action type, please try again.
Answer: Gesellschaft mit beschränkter Haftung
Thought: Canadian-American
Action: Canadian-American
Answer: Jonny" Craig
Thought: Maine gubernatorial election
Action: Maine gubernatorial election
Answer: Bath, Maine
Thought: Nick Park and Aardman Animations
Action: Nick Park and Aardman Animations
Answer: Creature Comforts
Thought: Naj
Action: Naj
Answer: fortnightly women interest magazine
Thought: Rome Protocols
Action: Rome Protocols
Answer: a failed coup attempt
Thought: Art of Anarchy
Action: Art of Anarchy
Answer: 2004
Thought: Love and Theft
Action: Love and Theft
Answer: Love and Theft
Thought: Southampton finished sev

#### Save the result log

In [27]:
with open(os.path.join(root, 'CoT', 'context', strategy.value, f'{len(agents)}_questions_{trial}_trials.txt'), 'w') as f:
    f.write(log)
save_agents(agents, os.path.join(root, 'CoT', 'context', strategy.value, 'agents'))